In [ ]:
!pip install scikit-learn==0.24.1  aif360==0.3.0  tensorflow==2.4.0 nodejs==0.1.1 ipywidgets==7.6.3 lime==0.2.0.1 import_ipynb==0.1.3 #aix360==0.2.1 

In [ ]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [1]:
import tensorflow as tf
from tensorflow import keras
import import_ipynb
from codait_utils import *

importing Jupyter notebook from codait_utils.ipynb


In [47]:
import os
target_column = os.environ.get('target_column','target')
image_column = os.environ.get('image_column','filename')
image_shape = os.environ.get('image_shape','400,400')
prediction_column = os.environ.get('prediction_column','prediction')
metadata = os.environ.get('metadata','metadata.csv')

In [3]:
def is_local():
    import socket
    return '172.17.0.2' == socket.gethostbyname(socket.gethostname())

In [4]:
if is_local():
    target_column = 'finding'

In [5]:
unzip('.','model.zip')
unzip('.','data.zip')

FileNotFoundError: [Errno 2] No such file or directory: 'data.zip'

In [6]:
model = keras.models.load_model('model')

In [17]:
import os.path  
import glob  
classes = glob.glob("data/*")
classes = map(lambda s: s.split('/')[1],classes)
classes = list(classes)
classes

['Pneumonia_Viral_Varicella',
 'Pneumonia_Viral_SARS',
 'Pneumonia_Bacterial_Mycoplasma',
 'Pneumonia_Bacterial_Staphylococcus_MRSA',
 'Unknown',
 'Pneumonia_Bacterial_Klebsiella',
 'Pneumonia_Viral_Herpes ',
 'Tuberculosis',
 'Pneumonia_Lipoid',
 'Pneumonia',
 'Pneumonia_Bacterial',
 'Pneumonia_Fungal_Aspergillosis',
 'Pneumonia_Bacterial_Legionella',
 'Pneumonia_Bacterial_Nocardia',
 'Pneumonia_Bacterial_E.Coli',
 'No Finding',
 'Pneumonia_Fungal_Pneumocystis',
 'Pneumonia_Viral_MERS-CoV',
 'Pneumonia_Bacterial_Streptococcus',
 'Pneumonia_Aspiration',
 'Pneumonia_Viral_COVID-19',
 'Pneumonia_Bacterial_Chlamydophila',
 'Pneumonia_Viral_Influenza_H1N1',
 'Pneumonia_Viral_Influenza',
 'todo']

In [7]:
import pandas as pd
df = pd.read_csv(metadata)

In [8]:
df

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29
0,2,0.0,M,65.0,Pneumonia_Viral_COVID-19,Y,Y,N,N,N,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,Pneumonia_Viral_COVID-19,Y,Y,N,N,N,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,Pneumonia_Viral_COVID-19,Y,Y,N,N,N,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,Pneumonia_Viral_COVID-19,Y,Y,N,N,N,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,Pneumonia_Viral_COVID-19,Y,NaN,N,N,N,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924,479,0.0,F,40.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,NaN,United Kingdom,images,072ecaf8c60a81980abb57150a8016_jumbo-9.jpeg,NaN,https://radiopaedia.org/cases/multifocal-round...,CC BY-NC-SA,Asthmatic. Shortness of breath and wheeze. Rou...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN
925,479,70.0,F,40.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,NaN,United Kingdom,images,ff33c406392b968d483174c97eb857_jumbo-9.jpeg,NaN,https://radiopaedia.org/cases/multifocal-round...,CC BY-NC-SA,Asthmatic. Shortness of breath and wheeze. The...,"Case courtesy of Dr Ian Bickle, Radiopaedia.or...",NaN
926,480,NaN,M,26.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,images,000001-266.jpg,NaN,https://www.eurorad.org/case/947,CC BY-NC-SA 4.0,fire-eater accidentally ingested a paraffin mi...,NaN,NaN
927,481,NaN,M,50.0,Pneumonia,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,images,000001-272.jpg,NaN,https://www.eurorad.org/case/934,CC BY-NC-SA 4.0,"The patient, a heavy smoker, was referred to t...",NaN,NaN


In [35]:
import numpy as np

dataset = []

for i, row in df.iterrows():    
    exec('target_size = ('+image_shape+')')
    image = tf.keras.preprocessing.image.load_img('data/'+row[target_column]+'/'+row[image_column],target_size=target_size)
    input_arr = keras.preprocessing.image.img_to_array(image)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
    predictions = model.predict(input_arr)
    prediction = predictions[0]
    prediction = classes[np.argmax(prediction)]
    dataset.append(prediction)


In [38]:
import numpy as np
dataset = np.array(dataset)

In [39]:
import pandas as pd
df = pd.read_csv(metadata)

In [41]:
df[prediction_column] = dataset.tolist()

In [46]:
df.to_csv(metadata, index=False)